In [14]:
from numpy import mean
from numpy import std
from numpy import dstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM
from tensorflow.keras.regularizers import l2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.metrics import *

In [15]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

4 Physical GPUs, 1 Logical GPU


In [3]:
s25 = np.load('s25.npy')
h25 = np.load('h25.npy')

In [4]:
s25_labels = np.array([0 for _ in range(0,len(s25))])
h25_labels = np.array([1 for _ in range(0,len(h25))])

In [5]:
X = np.append(s25,h25,axis=0)

In [6]:
y = np.append(s25_labels,h25_labels,axis=0)

In [7]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 30, 16
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation= 'relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [8]:
def summarize_results(accuracies,precisions,recalls,aucs):
    m, s = mean(accuracies), std(accuracies)
    print( ' Accuracy: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(precisions), std(precisions)
    print( ' Precision: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(recalls), std(recalls)
    print( ' Recall: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(aucs), std(aucs)
    print( ' AUC: %.3f%% (+/-%.3f) ' % (m, s))

In [9]:
def run_experiment(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [10]:
run_experiment(X,y)

Epoch 1/30
58/58 [==============================] - 9s 162ms/step - loss: 0.6929 - accuracy: 0.5542 - precision: 0.5303 - recall: 0.4299 - auc: 0.5750
Epoch 2/30
58/58 [==============================] - 9s 161ms/step - loss: 0.5963 - accuracy: 0.6583 - precision: 0.6401 - recall: 0.6192 - auc: 0.7418
Epoch 3/30
58/58 [==============================] - 9s 162ms/step - loss: 0.5320 - accuracy: 0.7360 - precision: 0.7101 - recall: 0.7383 - auc: 0.8106
Epoch 4/30
58/58 [==============================] - 9s 163ms/step - loss: 0.4853 - accuracy: 0.7820 - precision: 0.7596 - recall: 0.7827 - auc: 0.8471
Epoch 5/30
58/58 [==============================] - 9s 162ms/step - loss: 0.4132 - accuracy: 0.8094 - precision: 0.7873 - recall: 0.8131 - auc: 0.8950
Epoch 6/30
58/58 [==============================] - 9s 163ms/step - loss: 0.3318 - accuracy: 0.8620 - precision: 0.8561 - recall: 0.8481 - auc: 0.9356
Epoch 7/30
58/58 [==============================] - 9s 163ms/step - loss: 0.2693 - accuracy: 0

58/58 [==============================] - 9s 164ms/step - loss: 0.0361 - accuracy: 0.9869 - precision_1: 0.9929 - recall_1: 0.9790 - auc_1: 0.9993
Epoch 24/30
58/58 [==============================] - 10s 165ms/step - loss: 0.0298 - accuracy: 0.9934 - precision_1: 1.0000 - recall_1: 0.9860 - auc_1: 0.9992
Epoch 25/30
58/58 [==============================] - 10s 165ms/step - loss: 0.0278 - accuracy: 0.9901 - precision_1: 0.9929 - recall_1: 0.9860 - auc_1: 0.9995
Epoch 26/30
58/58 [==============================] - 10s 165ms/step - loss: 0.0385 - accuracy: 0.9923 - precision_1: 0.9907 - recall_1: 0.9930 - auc_1: 0.9973
Epoch 27/30
58/58 [==============================] - 9s 163ms/step - loss: 0.0510 - accuracy: 0.9901 - precision_1: 0.9906 - recall_1: 0.9883 - auc_1: 0.9969
Epoch 28/30
58/58 [==============================] - 10s 165ms/step - loss: 0.0537 - accuracy: 0.9847 - precision_1: 0.9814 - recall_1: 0.9860 - auc_1: 0.9972
Epoch 29/30
58/58 [==============================] - 10s 164

58/58 [==============================] - 10s 166ms/step - loss: 0.0592 - accuracy: 0.9759 - precision_3: 0.9656 - recall_3: 0.9836 - auc_3: 0.9983
Epoch 15/30
58/58 [==============================] - 10s 165ms/step - loss: 0.0759 - accuracy: 0.9726 - precision_3: 0.9786 - recall_3: 0.9626 - auc_3: 0.9953
Epoch 16/30
58/58 [==============================] - 10s 165ms/step - loss: 0.1310 - accuracy: 0.9529 - precision_3: 0.9425 - recall_3: 0.9579 - auc_3: 0.9889
Epoch 17/30
58/58 [==============================] - 10s 164ms/step - loss: 0.0748 - accuracy: 0.9737 - precision_3: 0.9720 - recall_3: 0.9720 - auc_3: 0.9969
Epoch 18/30
58/58 [==============================] - 10s 165ms/step - loss: 0.0594 - accuracy: 0.9803 - precision_3: 0.9812 - recall_3: 0.9766 - auc_3: 0.9981
Epoch 19/30
58/58 [==============================] - 10s 164ms/step - loss: 0.0429 - accuracy: 0.9836 - precision_3: 0.9836 - recall_3: 0.9813 - auc_3: 0.9989
Epoch 20/30
58/58 [==============================] - 10s 1

In [11]:
def evaluate_model_2(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 30, 16
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(LSTM(100, return_sequences=True, input_shape=(n_timesteps,n_features)))
    model.add(LSTM(50))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation= 'relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [16]:
def run_experiment_2(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model_2(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [17]:
run_experiment_2(X,y)

Epoch 1/30
58/58 [==============================] - 16s 283ms/step - loss: 0.6674 - accuracy: 0.5553 - precision_10: 0.5355 - recall_10: 0.3879 - auc_10: 0.6085
Epoch 2/30
58/58 [==============================] - 16s 284ms/step - loss: 0.5028 - accuracy: 0.7634 - precision_10: 0.7265 - recall_10: 0.7944 - auc_10: 0.8399
Epoch 3/30
58/58 [==============================] - 17s 286ms/step - loss: 0.3745 - accuracy: 0.8445 - precision_10: 0.8164 - recall_10: 0.8621 - auc_10: 0.9107
Epoch 4/30
58/58 [==============================] - 16s 284ms/step - loss: 0.2423 - accuracy: 0.9047 - precision_10: 0.8920 - recall_10: 0.9065 - auc_10: 0.9628
Epoch 5/30
58/58 [==============================] - 17s 285ms/step - loss: 0.1773 - accuracy: 0.9332 - precision_10: 0.9180 - recall_10: 0.9416 - auc_10: 0.9809
Epoch 6/30
58/58 [==============================] - 17s 286ms/step - loss: 0.1573 - accuracy: 0.9430 - precision_10: 0.9393 - recall_10: 0.9393 - auc_10: 0.9830
Epoch 7/30
58/58 [================

58/58 [==============================] - 17s 289ms/step - loss: 0.0122 - accuracy: 0.9989 - precision_11: 1.0000 - recall_11: 0.9977 - auc_11: 0.9987
Epoch 22/30
58/58 [==============================] - 17s 288ms/step - loss: 0.0111 - accuracy: 0.9989 - precision_11: 1.0000 - recall_11: 0.9977 - auc_11: 0.9987
Epoch 23/30
58/58 [==============================] - 17s 288ms/step - loss: 0.0085 - accuracy: 0.9989 - precision_11: 1.0000 - recall_11: 0.9977 - auc_11: 0.9987
Epoch 24/30
58/58 [==============================] - 17s 289ms/step - loss: 0.0061 - accuracy: 0.9989 - precision_11: 1.0000 - recall_11: 0.9977 - auc_11: 1.0000
Epoch 25/30
58/58 [==============================] - 17s 289ms/step - loss: 0.0087 - accuracy: 0.9989 - precision_11: 1.0000 - recall_11: 0.9977 - auc_11: 0.9987
Epoch 26/30
58/58 [==============================] - 17s 289ms/step - loss: 0.0126 - accuracy: 0.9989 - precision_11: 0.9977 - recall_11: 1.0000 - auc_11: 0.9989
Epoch 27/30
58/58 [=====================

58/58 [==============================] - 17s 287ms/step - loss: 0.0225 - accuracy: 0.9945 - precision_13: 0.9953 - recall_13: 0.9930 - auc_13: 0.9996
Epoch 12/30
58/58 [==============================] - 17s 288ms/step - loss: 0.0130 - accuracy: 0.9967 - precision_13: 0.9977 - recall_13: 0.9953 - auc_13: 0.9999
Epoch 13/30
58/58 [==============================] - 17s 290ms/step - loss: 0.0984 - accuracy: 0.9726 - precision_13: 0.9611 - recall_13: 0.9813 - auc_13: 0.9912
Epoch 14/30
58/58 [==============================] - 17s 288ms/step - loss: 0.0914 - accuracy: 0.9737 - precision_13: 0.9698 - recall_13: 0.9743 - auc_13: 0.9925
Epoch 15/30
58/58 [==============================] - 17s 288ms/step - loss: 0.0657 - accuracy: 0.9803 - precision_13: 0.9790 - recall_13: 0.9790 - auc_13: 0.9968
Epoch 16/30
58/58 [==============================] - 17s 287ms/step - loss: 0.0648 - accuracy: 0.9770 - precision_13: 0.9722 - recall_13: 0.9790 - auc_13: 0.9973
Epoch 17/30
58/58 [=====================